In [10]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210514T070063 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


In [11]:

training_data_uri = "s3://sagemaker-sample-data-{}/tensorflow/mnist".format(region)

In [12]:
training_data_uri

's3://sagemaker-sample-data-us-east-2/tensorflow/mnist'

In [16]:
!pygmentize 'mnist1.py'

import argparse
import json
import os

import numpy as np
import tensorflow as tf


def model(x_train, y_train, x_test, y_test):
    """Generate a simple model"""
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1024, activation=tf.nn.relu),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax),
        ]
    )

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    model.fit(x_train, y_train)
    model.evaluate(x_test, y_test)

    return model


def _load_training_data(base_dir):
    """Load MNIST training data"""
    x_train = np.load(os.path.join(base_dir, "train_data.npy"))
    y_train = np.load(os.path.join(base_dir, "train_labels.npy"))
    return x_train, y_train


def _load_testing_data(base_dir):
    """Load MNIST testing data"""
    x_test = np.load(os.path.join(base_dir, "eval_data.npy"))
    y_test = np

In [19]:
from sagemaker.tensorflow import TensorFlow


mnist_estimator = TensorFlow(
    entry_point="mnist1.py",
    role=role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    framework_version="2.1.0",
    py_version="py3",
    distribution={"parameter_server": {"enabled": True}},
)

In [20]:
mnist_estimator.fit(training_data_uri)

2021-05-14 12:09:02 Starting - Starting the training job...
2021-05-14 12:09:24 Starting - Launching requested ML instancesProfilerReport-1620994141: InProgress
......
2021-05-14 12:10:25 Starting - Preparing the instances for training......
2021-05-14 12:11:25 Downloading - Downloading input data...
2021-05-14 12:11:45 Training - Downloading the training image..2021-05-14 12:12:10,480 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-05-14 12:12:10,488 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-05-14 12:12:10,881 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-05-14 12:12:10,900 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-05-14 12:12:10,917 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-05-14 12:12:10,930 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework

In [22]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

In [23]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load("train_data.npy")
train_labels = np.load("train_labels.npy")

download: s3://sagemaker-sample-data-us-east-2/tensorflow/mnist/train_data.npy to ./train_data.npy
download: s3://sagemaker-sample-data-us-east-2/tensorflow/mnist/train_labels.npy to ./train_labels.npy


In [26]:
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = np.argmax(predictions["predictions"][i])
    label = train_labels[i]
    print(
        "prediction is {}, label is {}, matched: {}".format(prediction, label, prediction == label)
    )


prediction is 7, label is 7, matched: True
prediction is 3, label is 3, matched: True
prediction is 9, label is 4, matched: False
prediction is 6, label is 6, matched: True
prediction is 1, label is 1, matched: True
prediction is 8, label is 8, matched: True
prediction is 1, label is 1, matched: True
prediction is 0, label is 0, matched: True
prediction is 9, label is 9, matched: True
prediction is 8, label is 8, matched: True
prediction is 0, label is 0, matched: True
prediction is 3, label is 3, matched: True
prediction is 1, label is 1, matched: True
prediction is 3, label is 2, matched: False
prediction is 7, label is 7, matched: True
prediction is 0, label is 0, matched: True
prediction is 2, label is 2, matched: True
prediction is 9, label is 9, matched: True
prediction is 6, label is 6, matched: True
prediction is 0, label is 0, matched: True
prediction is 1, label is 1, matched: True
prediction is 6, label is 6, matched: True
prediction is 7, label is 7, matched: True
predictio

In [29]:
predictor.delete_endpoint()

ClientError: An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "arn:aws:sagemaker:us-east-2:496623956168:endpoint-config/tensorflow-training-2021-05-14-12-18-12-016".